<a href="https://colab.research.google.com/github/watcharap0n/m-business/blob/main/predict_product_each_day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

def change_database(path: str, key_transaction: str, start: int) -> list:
    with open(path, 'r') as json_file:
        file = json.load(json_file)
        customers = dict(file[key_transaction])
    id_data = list(customers)
    new_customers = []
    for i in id_data[start:]:
        data = customers[i]
        data['email_private'] = data.pop('emailliff')
        if data.get('month_check'):
            del data['month_check']
        if data.get('Tag'):
            del data['Tag']
        del data['authorized']
        del data['tax']
        del data['position']
        new_customers.append(data)
    return new_customers

data = change_database('/content/sample_data/chatbot-mango-export-3.json', 'RestCustomer', 5)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans


In [3]:
df = pd.DataFrame(data)
df = df.drop(['day', 'img', 'month', 'year', 'Tag', 'id', 'datetime', 'date_insert', 'time_insert', 'datetime_insert'], axis=1)
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%m/%d/%Y')
df

# drop dataframe and chang object to type datetime

,channel,company,date,email,message,name,other,picture,product,profile,tag,tel,time,username,email_private
0,event Impact,"Nimble suply and service co.,ltd",11/11/2020,assama82@gmail.com,"Asst.marketing manager\nบริษัททำ FA, LPS",อัสมา บุญเต็ม,,https://profile.line-scdn.net/0hV6AIt4J5CRl8MC...,Construction,Assama Boontem,,0632711038,13:6:27,tongta,assama82@gmail.com
1,event Impact,CNT,11/11/2020,raksheeth555@gmail.com,Planning and cost control,Raksheeth,,https://profile.line-scdn.net/0hSByirDrxDG1EEC...,Project Planning,N. Raksheeth,,0918847790,13:6:27,tongta,raksheeth555@gmail.com
2,event Impact,growcons,11/11/2020,ridtirud@gmail.com,ก่อสร้างบ้าน,เจริญชัย,,https://profile.line-scdn.net/0h4pgK2tBta19aMk...,Construction,เจริญชัย ฤทธิรุทธ🐎8,,0895164594,13:6:27,tongta,ridbetta@gmail.com
3,event Impact,กำลังจัดตั้ง,11/11/2020,bjakkapun@gmail.com,ceo,จักรพันธ์,,https://profile.line-scdn.net/0m0e7b86dd725152...,RealEstate,Chakkaphan B.,,0858216555,11:16:57,tongta,bjakkapun@gmail.com
4,event Impact,Pratommin,11/11/2020,pratomminsomkiat@gmail.com,Ceo,Somkiat,,https://profile.line-scdn.net/0hZjTQMseuBUVpKC...,Construction,Somkiat (Tony),,0866266375,11:16:57,tongta,sk@somkiatsafety.in.th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,GetDemo,Saengthong,07/19/2021,panadda.saengthong@gmail.com,ขอทราบรายละเอียดและราคาหน่อยค่ะ,Panadda,,,Mango ERP (Construction),,,0849403053,19:11:38,Maytika,
332,GetDemo,"SASICHAVIKIT Co.,ltd.",07/20/2021,contact@roovtect.com,,ชวิทย์ เสรีวัฒโนภาส,,,Mango ERP (Construction),,,0817204240,14:42:0,Maytika,
333,GetDemo,"SASICHAVIKIT Co.,ltd.",07/20/2021,contact@roovtect.com,,ชวิทย์ เสรีวัฒโนภาส,,,Mango ERP (Construction),,,0817204240,14:40:20,Maytika,
334,contact,ROOVTECT,07/20/2021,contact@roovtect.com,รบกวนให้ Sales ติดต่อครับ,Chavitya Seriwathanophas,,,Mango ERP (Construction),,,0817204240,14:40:20,Maytika,


In [4]:
df['channel'].loc[df['channel'] == 'contact'] = 'Contact'
df['product'].loc[df['product'] == 'Mango ERP (Construction)'] = 'Construction'
df['product'].loc[df['product'] == 'Mango ERP (Real Estate)'] = 'RealEstate'
df['product'].loc[df['product'] == 'Real Estates'] = 'RealEstate'
df['product'].loc[df['product'] == 'Pusit (Consulting)'] = 'Consulting'

events = df.loc[df['channel'] == 'event Impact']
df.drop(events.index, inplace=True)
df.info()


# drop channel of event Impact

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305 entries, 31 to 335
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   channel        305 non-null    object
 1   company        305 non-null    object
 2   date           305 non-null    object
 3   email          305 non-null    object
 4   message        305 non-null    object
 5   name           305 non-null    object
 6   other          305 non-null    object
 7   picture        305 non-null    object
 8   product        305 non-null    object
 9   profile        305 non-null    object
 10  tag            305 non-null    object
 11  tel            305 non-null    object
 12  time           305 non-null    object
 13  username       305 non-null    object
 14  email_private  305 non-null    object
dtypes: object(15)
memory usage: 38.1+ KB


In [6]:
event_realtech = df.loc[df['channel'] == 'Event Realtech']
facebook = df.loc[df['channel'] == 'facebook']

df.drop(event_realtech.index, inplace=True)
df.drop(facebook.index, inplace=True)

df = df.reset_index()
df

# cleaning delete channel 

,index,channel,company,date,email,message,name,other,picture,product,profile,tag,tel,time,username,email_private
0,31,LINE,Thaitharit,11/24/2020,sayompoo_i@thaitharit.co.th,None,สยมภู อิ่มระรื่น,,https://profile.line-scdn.net/0hQVyy-lTkDnlEAC...,Construction,'TE®,[CB010],0655126969,9:44:2,tongta,mvpter@gmail.com
1,32,LINE,หจก ภักดี การช่าง,05/12/2020,suwapa.ann@gmail.com,None,ศุวภาพ พาพาน,,https://profile.line-scdn.net/0hhm8JuH2jN0IFMx...,Construction,🍃🌸ANN🌸🍃,[CJ010],0924566916,6:43:28,Maytika,suwapa.ann@gmail.com
2,33,LINE,หจก.นาราภัทร คอนสตรัคชั่น,02/12/2020,bomzpobtron@hotmail.com,None,ภพธร ปราการัตน์,,https://profile.line-scdn.net/0hRV86b6guDXkFTy...,Construction,Pobtorn78,[CB010],0618829624,22:11:47,Maytika,bomzpobtron@hotmail.com
3,34,LINE,CNT,11/30/2020,piyachbo@cn-thai.co.th,None,ปิยชาติ โพธิทัต,,https://profile.line-scdn.net/0hTdKsU2tRC0t2Ci...,Construction,Piyachard Toy,,0814127756,9:50:54,Maytika,piyachard@hotmail.com
4,35,LINE,"Pasuk home 2465 .co.,ltd",10/12/2020,wuttikul28@gmail.com,None,Wuttikul,,https://profile.line-scdn.net/0hTu7JD8Y3C1oKMy...,Construction,PASUK HOME 2465,,0876894224,16:58:20,Kane,undefined
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,331,GetDemo,Saengthong,07/19/2021,panadda.saengthong@gmail.com,ขอทราบรายละเอียดและราคาหน่อยค่ะ,Panadda,,,Construction,,,0849403053,19:11:38,Maytika,
290,332,GetDemo,"SASICHAVIKIT Co.,ltd.",07/20/2021,contact@roovtect.com,,ชวิทย์ เสรีวัฒโนภาส,,,Construction,,,0817204240,14:42:0,Maytika,
291,333,GetDemo,"SASICHAVIKIT Co.,ltd.",07/20/2021,contact@roovtect.com,,ชวิทย์ เสรีวัฒโนภาส,,,Construction,,,0817204240,14:40:20,Maytika,
292,334,Contact,ROOVTECT,07/20/2021,contact@roovtect.com,รบกวนให้ Sales ติดต่อครับ,Chavitya Seriwathanophas,,,Construction,,,0817204240,14:40:20,Maytika,


In [7]:
print(set(df['product']))
print(set(df['channel']))



{'RealEstate', 'แบ่งชำระ เบา เบา', 'Construction', 'Other', 'Consulting', 'Project Planning', 'Mango Application on Mobile'}
{'GetDemo', 'LINE', 'Contact'}


In [8]:
df1 = df[['product', 'channel', 'date']]
# df1['date'] = pd.to_datetime(df1['date'])
# x['time'] = pd.to_datetime(x['time'], format='%H:%M:%S')
# x['date'] = [date.date() for date in x['date']]
# x['time'] = [time.time() for time in x['time']]

df1

,product,channel,date
0,Construction,LINE,11/24/2020
1,Construction,LINE,05/12/2020
2,Construction,LINE,02/12/2020
3,Construction,LINE,11/30/2020
4,Construction,LINE,10/12/2020
...,...,...,...
289,Construction,GetDemo,07/19/2021
290,Construction,GetDemo,07/20/2021
291,Construction,GetDemo,07/20/2021
292,Construction,Contact,07/20/2021


In [9]:
# iloc (row, col)
x = df1.reset_index()[['date']]

x['date'] = pd.to_datetime(df1['date'])
x['year'] = [x.iloc[i, 0].year for i in range(len(x))]
x['month'] = [x.iloc[i, 0].month for i in range(len(x))]
x['day'] = [x.iloc[i, 0].day for i in range(len(x))]
x['english_day'] = x.date.dt.strftime('%a')
x = x.drop(['date'], axis=1)

x
# split day, month, year from date

,year,month,day,english_day
0,2020,11,24,Tue
1,2020,5,12,Tue
2,2020,2,12,Wed
3,2020,11,30,Mon
4,2020,10,12,Mon
...,...,...,...,...
289,2021,7,19,Mon
290,2021,7,20,Tue
291,2021,7,20,Tue
292,2021,7,20,Tue


In [10]:
x = x.astype({
    'month': 'category',
    'day': 'category',
    'english_day': 'category'
    })

x.info()
# set change type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   year         294 non-null    int64   
 1   month        294 non-null    category
 2   day          294 non-null    category
 3   english_day  294 non-null    category
dtypes: category(3), int64(1)
memory usage: 5.5 KB


In [11]:
# x = x.merge(df1[['channel', 'product']], left_index=True, right_index=True).set_index(df['date'])
x = x.merge(df1[['channel']], left_index=True, right_index=True).set_index(df['date'])
x


,year,month,day,english_day,channel
date,,,,,
11/24/2020,2020,11,24,Tue,LINE
05/12/2020,2020,5,12,Tue,LINE
02/12/2020,2020,2,12,Wed,LINE
11/30/2020,2020,11,30,Mon,LINE
10/12/2020,2020,10,12,Mon,LINE
...,...,...,...,...,...
07/19/2021,2021,7,19,Mon,GetDemo
07/20/2021,2021,7,20,Tue,GetDemo
07/20/2021,2021,7,20,Tue,GetDemo


In [12]:
cat_df = x.select_dtypes(include=['category']).columns.to_list()
one_hot_df = pd.get_dummies(x, columns=cat_df)
one_hot_df

,year,channel,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,english_day_Fri,english_day_Mon,english_day_Sat,english_day_Sun,english_day_Thu,english_day_Tue,english_day_Wed
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11/24/2020,2020,LINE,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
05/12/2020,2020,LINE,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
02/12/2020,2020,LINE,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
11/30/2020,2020,LINE,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
10/12/2020,2020,LINE,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
07/19/2021,2021,GetDemo,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
07/20/2021,2021,GetDemo,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
07/20/2021,2021,GetDemo,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [14]:
class MultiColumnLabelEncoder:

    def __init__(self, columns=None):
        self.columns = columns # array of column names to encode


    def fit(self, X, y=None):
        self.encoders = {}
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            self.encoders[col] = LabelEncoder().fit(X[col])
        return self


    def transform(self, X):
        output = X.copy()
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
          output[col] = self.encoders[col].transform(X[col])
        return output


    def fit_transform(self, X, y=None):
        return self.fit(X,y).transform(X)


    def inverse_transform(self, X):
        output = X.copy()
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            output[col] = self.encoders[col].inverse_transform(X[col])
        return output

In [15]:
# multi = MultiColumnLabelEncoder(columns=['channel', 'product'])
multi = MultiColumnLabelEncoder(columns=['channel'])
X = multi.fit_transform(one_hot_df)
# display(set(X['product']))
display(set(X['channel']))
X

{0, 1, 2}

,year,channel,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,english_day_Fri,english_day_Mon,english_day_Sat,english_day_Sun,english_day_Thu,english_day_Tue,english_day_Wed
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11/24/2020,2020,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
05/12/2020,2020,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
02/12/2020,2020,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
11/30/2020,2020,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
10/12/2020,2020,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
07/19/2021,2021,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
07/20/2021,2021,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
07/20/2021,2021,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [16]:
inv = multi.inverse_transform(X)
# display(set(inv['product']))
display(set(inv['channel']))
inv

{'Contact', 'GetDemo', 'LINE'}

,year,channel,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,english_day_Fri,english_day_Mon,english_day_Sat,english_day_Sun,english_day_Thu,english_day_Tue,english_day_Wed
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11/24/2020,2020,LINE,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
05/12/2020,2020,LINE,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
02/12/2020,2020,LINE,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
11/30/2020,2020,LINE,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
10/12/2020,2020,LINE,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
07/19/2021,2021,GetDemo,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
07/20/2021,2021,GetDemo,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
07/20/2021,2021,GetDemo,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [17]:
# transform _y
le = LabelEncoder()
y = df1[['product']]
y = le.fit_transform(y)
display(set(le.inverse_transform(y)))
y

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'Construction',
 'Consulting',
 'Mango Application on Mobile',
 'Other',
 'Project Planning',
 'RealEstate',
 'แบ่งชำระ เบา เบา'}

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 1, 5, 0, 0, 0, 0, 0, 3, 2, 5,
       4, 0, 3, 5, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 5, 5, 0, 0, 0, 0, 5, 0, 0, 0, 0, 3, 0, 0, 0, 0, 5, 0,
       0, 0, 5, 5, 0, 5, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 5, 0, 2, 5, 6, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 5, 5, 0, 0, 0,
       5, 0, 0, 0, 0, 2, 2, 0, 0, 0, 5, 0, 5, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 5, 0, 5, 0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5,
       0, 0, 2, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 5, 2, 0, 0, 0, 0, 5, 5, 0,
       0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 5, 0, 0, 0, 4, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.4)


classifier = KNeighborsClassifier()
classifier.fit(x_train, y_train)
prediction = classifier.predict(x_test)
print('predict', prediction)
print('\n')
print('actual', y_test)

acc = accuracy_score(y_test, prediction)
print(acc)

predict [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


actual [0 0 0 0 0 0 0 0 0 3 0 0 5 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 3 0 0 0 2 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 2 0 0 0 0 0 0 0 0 0 5 5 0 5 2 0
 5 0 3 0 0 3 0 0 2 0 0 0 0 0 5 5 0 0 0 0 0 0 5 0 0 0 0 0 5 0 0 5 0 5 0 0 3
 0 0 0 0 0 0 5]
0.7966101694915254


In [22]:
!pip freeze > requirements.txt